In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from MNLP import MNLPTrainer

In [3]:
import sys
import os

In [4]:
import json

In [5]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from model import SimpleCNN

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'EGL')))

In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 10           # CIFAR-10
learning_rate = 5e-4

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class NewDataset(Dataset):
    def __init__(self, old_dataset, indexes):
        # Копируем данные на основе указанных индексов
        self.data = [old_dataset[idx] for idx in indexes]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


In [9]:
TRAIN_LEN = train_dataset.__len__()

In [ ]:
INITIAL_TRAIN = 50
p = INITIAL_TRAIN / train_dataset.__len__()
p # Начальный размер датасета

0.001

In [11]:
Trainer = MNLPTrainer

In [12]:
res = {}
for p in [0.01, 0.1, 0.2]:
    data_amount = int(TRAIN_LEN * p)
    init_train = data_amount // 10
    initial_indices = np.random.choice(len(train_dataset), size=init_train, replace=False)
    initial_data = Subset(train_dataset, initial_indices)
    unlabeled_indices = list(set(range(len(train_dataset))) - set(initial_indices))
    unlabeled_data = NewDataset(train_dataset, unlabeled_indices)


    train_dataloader = DataLoader(initial_data, batch_size=128, shuffle=True)
    val_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)
    pool_dataloader = DataLoader(unlabeled_data, batch_size=128, shuffle=False)

    print(f" Размер обучающего датасета {len(initial_data)}")
    print(f" Размер AL датасета {len(unlabeled_data)}")
    print(f" Размер тестового датасета {len(test_dataset)}")

    model = SimpleCNN(NUM_CLASSES).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate )
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    trainer = Trainer(model=model.to(DEVICE),
                optimizer=optimizer,
                criterion=criterion, 
                pool_loader=pool_dataloader, 
                train_loader=train_dataloader, 
                val_loader=val_dataloader, 
                device=DEVICE)
    
    _,_,f1 = trainer.fit(end_data_amaunt = data_amount, num_samples = init_train, epochs_for_batch = 5)
    print(f1)
    res[p] = f1
    torch.cuda.empty_cache()
    


 Размер обучающего датасета 50
 Размер AL датасета 49950
 Размер тестового датасета 10000


Training: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


Epoch 1/50 - Train data: 50
Train Loss: 2.3157


Training: 100%|██████████| 1/1 [00:00<00:00, 66.68it/s]


Epoch 2/50 - Train data: 50
Train Loss: 2.1981


...

Epoch 49/50 - Train data: 10000
Train Loss: 0.8123


Training: 100%|██████████| 79/79 [00:00<00:00, 121.32it/s]


Epoch 50/50 - Train data: 10000
Train Loss: 0.7792
[15413, 6044, 15355, 14231, 26312, 5501, 39347, 14360, 35383, 38388, 34129, 6383, 7347, 39660, 38211, 14293, 12273, 2285, 24947, 27743, 38436, 21860, 32802, 13970, 31569, 20285, 5872, 34298, 7992, 23653, 29709, 26058, 19324, 8699, 13057, 34563, 2346, 38437, 28624, 14813, 7926, 32723, 19643, 31981, 19306, 2241, 8671, 7866, 23301, 6837, 10376, 15361, 7117, 184, 17126, 15030, 32478, 5117, 33823, 33254, 39446, 28203, 22033, 5938, 29971, 4520, 18895, 33081, 14109, 34479, 3472, 11307, 19176, 22256, 25928, 25675, 34057, 14202, 15457, 34470, 15125, 9675, 3974, 2809, 34583, 11213, 14495, 17032, 12876, 31541, 11987, 16055, 33166, 25892, 19608, 5856, 21767, 26879, 3943, 33184, 4020, 32186, 12642, 13506, 5025, 7153, 22948, 23858, 10738, 13603, 8142, 34706, 20089, 3156, 28636, 13458, 284, 5495, 20608, 18474, 9855, 34804, 10302, 20242, 11315, 31764, 35724, 18191, 33457, 31310, 4901, 24171, 31961, 9781, 9751, 21539, 7098, 38923, 2042, 1374, 22812, 10

Validating: 100%|██████████| 79/79 [00:02<00:00, 38.44it/s]

Validation Loss: 1.1575, Accuracy: 0.6227, F1 Score: 0.6212
0.6212453018742381


In [13]:
def save_f1_to_json(method_name, res, file_path="results.json"):
    # Создаем данные для текущего метода


    # Если файл существует, открываем и загружаем данные
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = {}  # Если файл пустой или некорректный
    else:
        data = {}

    # Обновляем или добавляем данные для текущего метода
    data[method_name] = res

    # Записываем обновленные данные обратно в файл
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

In [14]:
res

{0.01: 0.30365028184774145, 0.1: 0.5537896556420806, 0.2: 0.6212453018742381}

In [15]:
method_name = "MNLP"  # Имя текущего метода

# Сохраняем данные
save_f1_to_json(method_name, res)